In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import gurobipy as gp 
from gurobipy import GRB
import os

np.random.seed(2023)

In [2]:
def generate_A_matrix(n):
    # 创建基本矩阵块
    I_n = np.eye(n)        # n×n 单位矩阵
    zero_block = np.zeros((n, n))  # n×n 零矩阵
    
    # 创建每一行块
    # row1 = np.hstack([I_n, -I_n, I_n])      # 第一行: [I_n, -I_n, I_n]
    # row2 = np.hstack([zero_block, -I_n, zero_block])  # 第二行: [0, -I_n, 0]
    # row3 = np.hstack([zero_block, zero_block, -I_n])  # 第三行: [0, 0, -I_n]
    
    # 垂直堆叠所有行块
    block_matrix = np.block([[I_n, -I_n, I_n], [zero_block, -I_n, zero_block], [zero_block, zero_block, -I_n]])
    
    return block_matrix

def hat_f(x, X, b):
    return np.linalg.norm(X@x-b, 2)**2/2.0

def grad_hat_f(x, X, b):
    return X.T@(X@x-b)

def f(x, X, b, lambd, n):
    hat_x = x[:n]
    bar_x = x[n:]
    return np.linalg.norm(X@hat_x-b, 2)**2/2.0+lambd * np.ones(2*n)@bar_x

def grad_f(x, X, b, lambd, n):
    hat_x = x[:n]
    bar_x = x[n:]
    hat_grad = X.T@(X@hat_x-b)
    bar_grad = lambd * np.ones(2*n)
    return np.concatenate([hat_grad, bar_grad], axis=0) 

In [3]:
# parameters
'''
n: dimension of the primal variable
m: dimension of the dual variable

lf: smoothness parameter of the objective function
mf: strong convexity parameter of the objective function

alpha: the parameter multiplying the constraint matrix
'''

n = 5
m = n
In = np.eye(n)
Im = np.eye(m)
Bdim = 3*n
dim = 6*n
time = 800
step = 2000
time_eval = np.linspace(0,time,step)

A = generate_A_matrix(n)
large_sigma_A = np.max(np.linalg.eigvals(A @ A.T))
small_sigma_A = np.min(np.linalg.eigvals(A @ A.T))      


try:
    os.makedirs('Para', exist_ok=True)
    
    X = np.load('Para/matrix_X.npy')
    b = np.load('Para/vector_b.npy')
    w0 = np.load('Para/vector_x0.npy')
    lambd = np.load('Para/lambd.npy')

    lf = np.linalg.eigvalsh(X @ X.T).max()
    mf = np.linalg.eigvalsh(X @ X.T).min()
    
except FileNotFoundError:
    print("Error:Not Find Parameters")

B = np.eye(3*n)
large_sigma_B = np.max(np.linalg.eigvals(B @ B.T))
small_sigma_B = np.min(np.linalg.eigvals(B @ B.T))

alpha = 4.0/lf
eta = 1.0
rho = 1.0
print(large_sigma_A,small_sigma_A,large_sigma_B,small_sigma_B, lf, mf, b, w0)

3.7320508075688767 0.2679491924311221 1.0 1.0 16.491425542730166 1.623868484603545e-05 [-1.10766342 -1.94602338 -1.48254958 -2.20816044 -0.49962592] [0.79162115 0.8103383  0.98055723 0.88478525 0.10980113 0.81971076
 0.30761289 0.26149467 0.40572354 0.55342038 0.62552644 0.07876025
 0.97228343 0.41131105 0.7216644  0.66328748 0.21822526 0.18717254
 0.72977924 0.86331326 0.39172036 0.11004811 0.9127915  0.35700599
 0.41296218 0.18354969 0.58599027 0.85567085 0.78968122 0.08784242]


In [4]:


# compute the optimal solution to the equivalent form of Lasso regression problem
def gurobi_transformed_lasso(X, b, lambd):
    dim = 3*n
    model = gp.Model("gp")
    model.setParam('OutputFlag', 0)
    model.Params.Threads = 0 
    u = model.addMVar(dim, lb=-gp.GRB.INFINITY, name='u')


    hat_x = u[:n]
    bar_x = u[n:]
    obj = 0.5 * ((X@hat_x-b)@(X@hat_x-b)).sum() + lambd*np.ones(2*n)@bar_x
    # set objective
    model.setObjective(obj, GRB.MINIMIZE)
    # set constraints
    I = np.eye(n)
    # B = np.block([-I, I])
    model.addConstr(hat_x - u[n:2*n] + u[2*n:3*n] == 0)
    model.addConstr(u[n:] >= 0)

    model.update()
    model.optimize()

    u_opt = np.zeros(dim)
    v_opt = np.zeros(dim)
    if model.status == GRB.OPTIMAL:
        # Access the optimal variable values
        for i in range(dim):
            u_opt[i] = model.getVarByName('u[{}]'.format(i)).X
    else:
        print("Optimization did not converge to an optimal solution.")
    flag = 0
    for con in model.getConstrs():
        v_opt[flag] = con.Pi
        flag += 1
    v_opt[:n] = -v_opt[:n]
    return model.objVal, u_opt, v_opt

f_opt, x_opt, y_opt = gurobi_transformed_lasso(X,b,lambd)
print(x_opt,"\n", y_opt)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-19
[-2.83697848e-08 -5.89781782e-01 -3.19751533e-09 -2.43868534e-10
 -7.96403353e-11  2.79790601e-12  2.79717345e-12  2.89320527e-12
  3.10171256e-12  3.29209308e-12  2.83725832e-08  5.89781782e-01
  3.20040895e-09  2.46971604e-10  8.29324290e-11] 
 [-3.95788639 -4.         -3.63690454 -2.865819   -3.44762093  7.95788639
  8.          7.63690454  6.865819    7.44762093  0.04211361  0.
  0.36309546  1.134181    0.55237907]


In [5]:


# compute the optimal solution to the equivalent form of Lasso regression problem
def LeastSquare(X,b):
    dim = n
    model = gp.Model("gp")
    model.setParam('OutputFlag', 0)
    model.Params.Threads = 0 
    u = model.addMVar(dim, lb=-gp.GRB.INFINITY, name='u')

    obj = 0.5 * ((X@u-b)@(X@u-b)).sum() 
    model.setObjective(obj, GRB.MINIMIZE)

    model.update()
    model.optimize()

    u_opt = np.zeros(dim)
    v_opt = np.zeros(dim)
    if model.status == GRB.OPTIMAL:
        # Access the optimal variable values
        for i in range(dim):
            u_opt[i] = model.getVarByName('u[{}]'.format(i)).X
    else:
        print("Optimization did not converge to an optimal solution.")
    flag = 0
    for con in model.getConstrs():
        v_opt[flag] = con.Pi
        flag += 1
    v_opt[:n] = -v_opt[:n]
    return model.objVal, u_opt, v_opt

hat_f_opt, hat_x_opt, hat_y_opt = LeastSquare(X,b)
print(hat_x_opt,"\n", hat_y_opt)

[-116.59547707  -29.59849378  177.76123521   74.0178565   -83.33830612] 
 [-0. -0. -0. -0. -0.]


In [6]:
# Compute the intermediate variable p_star
def cal_p_star(A, b, X, B, u, v, alpha, rho):
    dim =3*n
    B_inv = np.linalg.inv(B)
    model = gp.Model('qp')
    model.setParam('OutputFlag', 0)
    model.Params.Threads = 0 
    p = model.addMVar(dim, lb=-GRB.INFINITY, name='p')
    model.addConstr(p[n:] >= 0)
    
    tilde_x = u-alpha*A.T@B_inv@p
    hat_x = tilde_x[:n]
    bar_x = tilde_x[n:]
    obj = 0
    obj += 0.5 * ((X @ hat_x - b) @ (X @ hat_x - b)) + lambd * (np.ones(2*n) @ bar_x)                  # linear term
    obj +=  p @ (B_inv.T @ A @ u)                # linear term
    obj += - alpha * ((A.T @ B_inv @ p) @ (A.T @ B_inv @ p))  # quadratic term
    obj += - rho/2.0 * ((p - v) @ (p - v))                 # quadratic term

    # Add the scalar f_val to the Gurobi expression
    model.setObjective(obj, GRB.MAXIMIZE)
    model.optimize()

    p_opt = p.x

    return p_opt

In [7]:
# dynamics
'''
'''
def dynamic(x, t, alpha, rho):
    u,v = x[:3*n],x[3*n:]
    p_star = cal_p_star(A, b, X, B, u, v, alpha, rho)
    # print(p_star)
    B_inv = np.linalg.inv(B)

    du = - grad_f(u-alpha*A.T@B_inv@p_star, X, b, lambd, n) - A.T@B_inv@p_star
    dv = rho*(p_star-v)

    # \bar_y should be non-positive
    for i in range(2*n):
        if v[n+i] <= 0:
            dv[n+i] = max(0, dv[n+i])
    dynamics = np.concatenate((du, dv)).reshape(6*n).tolist()
    # print(dynamics)
    return dynamics

In [8]:
# results
res = []
t = np.linspace(0,time,step)
res = odeint(dynamic, w0, t, args=(alpha, rho))

In [9]:
x_orig_opt = LeastSquare(X,b)
f_opt, x_opt, y_opt= gurobi_transformed_lasso(X, b, lambd)
solution =res[-1]
u_opt = solution[:3*n]
v_opt = solution[3*n:]
x = u_opt - alpha * A.T @ np.linalg.inv(B) @ v_opt
y = np.linalg.inv(B) @ v_opt
# Double check 
print('x:',x)
print('y:',y)
print('x_orig_opt',x_orig_opt)
print('x_opt:',x_opt)
print('y_opt:',y_opt)
print("f(x): ", f(x, X, b, lambd, n))
print("f_opt: ", f(x_opt, X, b, lambd, n))

x: [-1.67174970e-07 -5.89781649e-01 -7.33273442e-11 -2.04691819e-11
 -4.67829109e-11  3.59257069e-12  3.51851881e-12  3.72901710e-12
  4.12536671e-12  3.81517040e-12  1.67178645e-07  5.89781649e-01
  7.71116504e-11  2.46259679e-11  5.06409359e-11]
y: [-3.95788637e+00 -4.00000000e+00 -3.63690453e+00 -2.86581899e+00
 -3.44762092e+00  7.95788637e+00  8.00000000e+00  7.63690453e+00
  6.86581899e+00  7.44762092e+00  4.21136309e-02  4.73656478e-11
  3.63095467e-01  1.13418101e+00  5.52379077e-01]
x_orig_opt (-7.403270201322698e-06, array([-116.59547707,  -29.59849378,  177.76123521,   74.0178565 ,
        -83.33830612]), array([-0., -0., -0., -0., -0.]))
x_opt: [-2.83697848e-08 -5.89781782e-01 -3.19751533e-09 -2.43868534e-10
 -7.96403353e-11  2.79790601e-12  2.79717345e-12  2.89320527e-12
  3.10171256e-12  3.29209308e-12  2.83725832e-08  5.89781782e-01
  3.20040895e-09  2.46971604e-10  8.29324290e-11]
y_opt: [-3.95788639 -4.         -3.63690454 -2.865819   -3.44762093  7.95788639
  8.       

In [10]:
import pickle
now = datetime.datetime.now()
# Pickling the object
Storage = {'result':res,'rho': rho, 'alpha': alpha, 'lambd': lambd,'X_matrix': X, 'time': time, 'step': step, 'time_step': time_eval, 'n': n, 'x_opt': x_opt, 'y_opt': y_opt, 'l':lf}
with open('Result/results_{}_parameters.pkl'.format(now), 'wb') as file:
    pickle.dump(Storage, file)
